In [1]:
#### download prediction model for lab of origin
#### other models can also be found in the prediction_models folder

!wget https://huggingface.co/lingxusb/PlasmidGPT/resolve/main/embedding_prediction_labs.pth
!wget https://huggingface.co/lingxusb/PlasmidGPT/resolve/main/lab_list.txt

--2024-11-14 12:19:37--  https://huggingface.co/lingxusb/PlasmidGPT/resolve/main/embedding_prediction_labs.pth
Resolving huggingface.co (huggingface.co)... 2600:9000:275b:2600:17:b174:6d00:93a1, 2600:9000:275b:e800:17:b174:6d00:93a1, 2600:9000:275b:f000:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:275b:2600:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/39/1b/391b3344dcaf5b835fcc8bdaa650824a769edb190566148b45b4de13176e8be8/1466554a6485bd7a5cc9c951d7d2c109f84c3cfe2a21322f826eb9621c274f22?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27embedding_prediction_labs.pth%3B+filename%3D%22embedding_prediction_labs.pth%22%3B&Expires=1731842378&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTg0MjM3OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzM5LzFiLzM5MWIzMzQ0ZGNhZjViODM1ZmNjOGJkYWE2NTA4MjRhNzY5ZWRiMTkw

In [1]:
import torch
import numpy as np
from transformers import PreTrainedTokenizerFast
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd

In [2]:
train_values_df = pd.read_csv('../data/train_values.csv')
train_labels_df = pd.read_csv('../data/train_labels.csv')

In [13]:
# Define the lab map
labs_name = {
    "R5B3KVZI": "Omar Akbari",
    "0PJ91ZT6": "Edward Boyden",
    "AAURK3RG": "Jim Haseloff",
    "FHZYKEUV": "Chris Voigt"
}

# Initialize a list to store DataFrames for each lab
filtered_dfs = []

# Loop over each lab key and filter the samples for that lab
for lab_key, lab_name in labs_name.items():
    if lab_key in train_labels_df.columns:
        # Select rows where the lab column has a value of 1
        lab_samples = train_labels_df[train_labels_df[lab_key] == 1]
        
        # Merge with train_values_df to get the sequences for these samples
        merged_df = pd.merge(lab_samples[['sequence_id']], train_values_df, on='sequence_id', how='inner')
        
        # Optionally, add a column for lab name for easy identification
        merged_df['lab_key'] = lab_key
        merged_df['lab_name'] = lab_name
        
        # Append to the list
        filtered_dfs.append(merged_df)
    else:
        print(f"Lab key '{lab_key}' not found in train_labels_df columns.")

# Concatenate all filtered DataFrames into one, if needed
all_labs_df = pd.concat(filtered_dfs, ignore_index=True)

# Display the combined filtered data
print(all_labs_df)


    sequence_id                                           sequence  \
0         QA9IF  ATCTTGAAGTTCACCTTGATGCCGTTCTTCTGCTTGTCGGCCATGA...   
1         IPO0O  ATCTTGAAGTTCACCTTGATGCCGTTCTTCTGCTTGTCGGCCATGA...   
2         3YF49  GATATAGACGTTGTGGCTGTTGTAGTTGTACTCCAGCTTGTGCCCC...   
3         TVCX7  ACCTACACCGAACTGAGATACCTACAGCGTGAGCTATGAGAAAGCG...   
4         3UXCC  GAGAAAGCGCCACGCTTCCCGAAGGGAGAAAGGCGGACAGGTATCC...   
..          ...                                                ...   
451       VV386  GGGAGGTGTCTTCGGCGCGCCTTTACGGCTAGCTCAGTCCTAGGTA...   
452       IKVOC  GGGAGGTGTCTTCGGCGCGCCTTTACGGCTAGCTCAGTCCTAGGTA...   
453       14HFH  GAAGACACGGAGTTCAGCCAAAAAACTTAAGACCGCCGGTCTTGTC...   
454       0FY10  CGGTCGAGGCGGTGCACGCGTGGCGCAATGCGCTCACGGGAGCACC...   
455       EWM56  GGAGCTCGGTACCAAATTCCAGAAAAGAGGCCTCCCGAAAGGGGGG...   

     bacterial_resistance_ampicillin  bacterial_resistance_chloramphenicol  \
0                                1.0                                   0.0   
1  

In [14]:
print(all_labs_df.columns)

Index(['sequence_id', 'sequence', 'bacterial_resistance_ampicillin',
       'bacterial_resistance_chloramphenicol',
       'bacterial_resistance_kanamycin', 'bacterial_resistance_other',
       'bacterial_resistance_spectinomycin', 'copy_number_high_copy',
       'copy_number_low_copy', 'copy_number_unknown',
       'growth_strain_ccdb_survival', 'growth_strain_dh10b',
       'growth_strain_dh5alpha', 'growth_strain_neb_stable',
       'growth_strain_other', 'growth_strain_stbl3', 'growth_strain_top10',
       'growth_strain_xl1_blue', 'growth_temp_30', 'growth_temp_37',
       'growth_temp_other', 'selectable_markers_blasticidin',
       'selectable_markers_his3', 'selectable_markers_hygromycin',
       'selectable_markers_leu2', 'selectable_markers_neomycin',
       'selectable_markers_other', 'selectable_markers_puromycin',
       'selectable_markers_trp1', 'selectable_markers_ura3',
       'selectable_markers_zeocin', 'species_budding_yeast', 'species_fly',
       'species_human', 

In [15]:
all_labs_df_2k = all_labs_df[all_labs_df['sequence'].apply(len) >= 2000].reset_index(drop=True)

In [11]:
# Load the pre-trained model and tokenizer
pt_file_path = 'pretrained_model.pt'
model = torch.load(pt_file_path, map_location=torch.device('cpu'))
model.eval()
print("Model loaded successfully.")
tokenizer = PreTrainedTokenizerFast(tokenizer_file='addgene_trained_dna_tokenizer.json')

# Add special tokens
special_tokens_dict = {'additional_special_tokens': ['[PROMPT]', '[PROMPT2]']}
tokenizer.add_special_tokens(special_tokens_dict)


# Set the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using device: {torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "Using CPU.")


Model loaded successfully.
Using CPU.


In [16]:
# Load the neural network for prediction
class SimpleNN(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# number of unique lab labels is 948
num_classes = 948

# Why the input dimension is this? 
input_dim = 768

model_path = "embedding_prediction_labs.pth"
list_file_name = 'lab_list.txt'

model_NN = SimpleNN(input_dim, num_classes).to(device)
model_NN.load_state_dict(torch.load(model_path, map_location=device))
model_NN.eval()

SimpleNN(
  (fc1): Linear(in_features=768, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=948, bias=True)
)

In [17]:
# Load labels (lab names)
lab_list = []
with open(list_file_name, 'r') as file:
    lab_list = [line.strip() for line in file]
lab_list = np.array(lab_list)

# Initialize list to store predicted labs
predicted_labs = []

# Loop over the first 5 sequences to predict the lab
for idx, row in all_labs_df_2k.head(5).iterrows():  # Use .head(5) to limit the rows to the first 5
    sequence = row['sequence']
    
    # Tokenize the sequence
    tokenized_sequence = [3]*10 + [2] + tokenizer.encode(sequence)
    tokenized_sequence = tokenized_sequence[:512]
    
    # Get embeddings from the model
    input_ids = torch.tensor([tokenized_sequence], dtype=torch.long).to(device)
    model.config.output_hidden_states = True
    
    # Inference to obtain hidden states
    with torch.no_grad():
        outputs = model(input_ids)
        hidden_states = outputs.hidden_states[-1].cpu().numpy()
        hidden_states_mean = np.mean(hidden_states, axis=1).reshape(-1)
    
    # Perform inference using the neural network
    X_data = torch.tensor([hidden_states_mean], dtype=torch.float32).to(device)
    output = model_NN(X_data)
    
    # Calculate probabilities using softmax
    probs = torch.nn.functional.softmax(output, dim=1).detach().cpu().numpy()[0]
    
    # Get the most likely lab
    predicted_lab_idx = np.argmax(probs)
    predicted_lab = lab_list[predicted_lab_idx]
    
    # Append the predicted lab to the list
    predicted_labs.append(predicted_lab)

# Ensure the dataframe only contains the first 5 rows
all_labs_df_2k_first_5 = all_labs_df_2k.head(5)

# Add the predicted labs to the dataframe
all_labs_df_2k_first_5['predicted_lab_name'] = predicted_labs

# Show the first few rows of the dataframe with the new column
print(all_labs_df_2k_first_5)

# Optionally, you can save the dataframe with predictions to a new CSV file
all_labs_df_2k_first_5.to_csv('all_labs_df_2k_first_5_with_predictions.csv', index=False)


<ipython-input-17-fb9402d0627e>:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)
  X_data = torch.tensor([hidden_states_mean], dtype=torch.float32).to(device)


  sequence_id                                           sequence  \
0       QA9IF  ATCTTGAAGTTCACCTTGATGCCGTTCTTCTGCTTGTCGGCCATGA...   
1       IPO0O  ATCTTGAAGTTCACCTTGATGCCGTTCTTCTGCTTGTCGGCCATGA...   
2       3YF49  GATATAGACGTTGTGGCTGTTGTAGTTGTACTCCAGCTTGTGCCCC...   
3       TVCX7  ACCTACACCGAACTGAGATACCTACAGCGTGAGCTATGAGAAAGCG...   
4       3UXCC  GAGAAAGCGCCACGCTTCCCGAAGGGAGAAAGGCGGACAGGTATCC...   

   bacterial_resistance_ampicillin  bacterial_resistance_chloramphenicol  \
0                              1.0                                   0.0   
1                              1.0                                   0.0   
2                              1.0                                   0.0   
3                              1.0                                   0.0   
4                              1.0                                   0.0   

   bacterial_resistance_kanamycin  bacterial_resistance_other  \
0                             0.0                         0.0   
1   

<ipython-input-17-fb9402d0627e>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_labs_df_2k_first_5['predicted_lab_name'] = predicted_labs


In [ ]:
from collections import Counter

# Initialize a dictionary to track tokens and their corresponding segments
token_segment_map = {}

# Counter to track token frequencies
token_counter = Counter()

# Specify the lab name for which you want to analyze sequences
target_lab = "Chris Voigt"

# Filter sequences belonging to the target lab
target_lab_sequences = all_labs_df_2k[all_labs_df_2k['lab_name'] == target_lab]['sequence']

# Loop over sequences and tokenize
for sequence in target_lab_sequences:
    # Tokenize the sequence
    tokenized_sequence = [3]*10 + [2] + tokenizer.encode(sequence)
    tokenized_sequence = tokenized_sequence[:512]
    
    # Update the token counter
    token_counter.update(tokenized_sequence)
    
    # Map each token to its sequence segment
    for token in tokenized_sequence:
        if token not in token_segment_map:
            segment = tokenizer.decode([token])  # Decode the token to get the corresponding sequence segment
            token_segment_map[token] = segment

# Convert the token counter to a DataFrame
token_freq_df = pd.DataFrame(token_counter.items(), columns=['Token', 'Frequency'])

# Add the sequence segment corresponding to each token
token_freq_df['Sequence Segment'] = token_freq_df['Token'].map(token_segment_map)

# Sort by frequency in descending order
token_freq_df = token_freq_df.sort_values(by='Frequency', ascending=False)

# Display the top recurring tokens and their corresponding segments
print(token_freq_df.head(20))

# Optionally save to CSV
token_freq_df.to_csv('token_frequencies_with_segments.csv', index=False)


     Token  Frequency Sequence Segment
0        3       3150            [PAD]
33      37        364              GTA
17      30        360              TAA
432     81        351             TTAA
591     53        345             TGTT
24      54        344             GAAA
118     20        342              TGC
323     27        324              GTT
282     48        322             TGCA
1        2        315            [SEP]
319     29        314              GCA
581     47        307             TGAA
14      40        305              GTC
322    114        285             GGTC
546     34        283              TTC
697    104        282             GCAC
195     89        270             TGTA
460    131        263            TGAAA
571     24        249              AAA
54      25        246              TGA


In [27]:
from collections import Counter

# Initialize a dictionary to track tokens and their corresponding segments
token_segment_map = {}

# Counter to track token frequencies
token_counter = Counter()

# Specify the lab name for which you want to analyze sequences
target_lab = "Jim Haseloff"

# Filter sequences belonging to the target lab
target_lab_sequences = all_labs_df_2k[all_labs_df_2k['lab_name'] == target_lab]['sequence']

# Loop over sequences and tokenize
for sequence in target_lab_sequences:
    # Tokenize the sequence
    tokenized_sequence = [3]*10 + [2] + tokenizer.encode(sequence)
    tokenized_sequence = tokenized_sequence[:512]
    
    # Update the token counter
    token_counter.update(tokenized_sequence)
    
    # Map each token to its sequence segment
    for token in tokenized_sequence:
        if token not in token_segment_map:
            segment = tokenizer.decode([token])  # Decode the token to get the corresponding sequence segment
            token_segment_map[token] = segment

# Convert the token counter to a DataFrame
token_freq_df = pd.DataFrame(token_counter.items(), columns=['Token', 'Frequency'])

# Add the sequence segment corresponding to each token
token_freq_df['Sequence Segment'] = token_freq_df['Token'].map(token_segment_map)

# Sort by frequency in descending order
token_freq_df = token_freq_df.sort_values(by='Frequency', ascending=False)

# Display the top recurring tokens and their corresponding segments
print(token_freq_df.head(20))

# Optionally save to CSV
token_freq_df.to_csv('token_frequencies_with_segments.csv', index=False)


     Token  Frequency Sequence Segment
0        3        260            [PAD]
2       30         57              TAA
144    137         57             TCTA
222     90         54             TAAC
32      75         53             GTAA
9       23         44              TGG
151     97         44             GTTC
23      39         43              GAC
143     42         42             TGGC
121     37         40              GTA
45     118         40             TCTC
209    176         37            TGGTT
363     93         37             GAAC
158    104         36             GCAC
67      34         35              TTC
215     26         35              TCA
233     16         34               TC
22     237         33            TGCAC
547     89         33             TGTA
447     41         33              TAC


In [42]:
# Filter sequences for the target lab and other labs
target_lab = "Chris Voigt"
target_lab_sequences = all_labs_df_2k[all_labs_df_2k['lab_name'] == target_lab]['sequence']
other_labs_sequences = all_labs_df_2k[all_labs_df_2k['lab_name'] != target_lab]['sequence']

# Token frequency for the target lab
target_token_counter = Counter()
for sequence in target_lab_sequences:
    tokenized_sequence = [3]*10 + [2] + tokenizer.encode(sequence)
    tokenized_sequence = tokenized_sequence[:512]
    target_token_counter.update(tokenized_sequence)

# Token frequency for other labs
other_token_counter = Counter()
for sequence in other_labs_sequences:
    tokenized_sequence = [3]*10 + [2] + tokenizer.encode(sequence)
    tokenized_sequence = tokenized_sequence[:512]
    other_token_counter.update(tokenized_sequence)

# Compare frequencies
target_freq_df = pd.DataFrame(target_token_counter.items(), columns=['Token', 'Frequency_Target'])
other_freq_df = pd.DataFrame(other_token_counter.items(), columns=['Token', 'Frequency_Other'])
freq_comparison = pd.merge(target_freq_df, other_freq_df, on='Token', how='outer').fillna(0)
freq_comparison['Target_Specificity'] = freq_comparison['Frequency_Target'] / (
    freq_comparison['Frequency_Other'] + 1e-5)  # Avoid division by zero

# Sort by target specificity
freq_comparison = freq_comparison.sort_values(by='Target_Specificity', ascending=False)

print(freq_comparison.head(10))
# Get the top 10 tokens with the highest specificity
top_tokens = freq_comparison.head(10)['Token'].tolist()

# Function to map tokens back to DNA segments
def get_sequence_segment_for_token(token, sequences, tokenizer):
    segments = []
    for sequence in sequences:
        # Tokenize the sequence
        tokenized_sequence = [3]*10 + [2] + tokenizer.encode(sequence)
        tokenized_sequence = tokenized_sequence[:512]
        
        # Check if the token is in the sequence
        if token in tokenized_sequence:
            # Decode the token to find the corresponding segment
            decoded_segment = tokenizer.decode([token])
            segments.append((sequence, decoded_segment))
    
    return segments

# Get the sequence segments for each top token
token_to_segments = {}
for token in top_tokens:
    token_to_segments[token] = get_sequence_segment_for_token(token, target_lab_sequences, tokenizer)

# Map tokens to their sequence segments
token_segment_map = {}

# Map all unique tokens to their segments
for token in freq_comparison['Token']:
    if token not in token_segment_map:
        # Decode the token to get the corresponding sequence segment
        token_segment_map[token] = tokenizer.decode([token])

# Add a new column for token representation
freq_comparison['Token_Representation'] = freq_comparison['Token'].map(token_segment_map)

# Set pandas options to display the full content of each cell
pd.set_option('display.max_colwidth', None)  # Ensure no truncation of column content
pd.set_option('display.max_rows', None)      # Optional: Display all rows if needed
pd.set_option('display.max_columns', None)   # Optional: Display all columns if needed

# Display the updated DataFrame with full sequence segments
print(freq_comparison.head(10))  # Adjust this to show more rows if necessary


      Token  Frequency_Target  Frequency_Other  Target_Specificity
2317  27329             101.0              0.0          10100000.0
2320  12285             101.0              0.0          10100000.0
2312  14029             101.0              0.0          10100000.0
2315   9957             101.0              0.0          10100000.0
2316  20319             101.0              0.0          10100000.0
2319  16070             101.0              0.0          10100000.0
1668    447              93.0              0.0           9300000.0
1632    851              83.0              0.0           8300000.0
3810  29038              73.0              0.0           7300000.0
3809  27502              68.0              0.0           6800000.0
      Token  Frequency_Target  Frequency_Other  Target_Specificity  \
2317  27329             101.0              0.0          10100000.0   
2320  12285             101.0              0.0          10100000.0   
2312  14029             101.0              0.0       

2. Token Contribution to Model Prediction

- To Evaluate the contribution of each token to the model's output for the target lab.
- Mask or remove tokens from the sequence and observe changes in the prediction probability.